In [ ]:
import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import math
import scipy
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import tqdm
from IPython.display import clear_output

sns.set_theme(style='darkgrid')

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x, y, t, u, v, p):
        super(CustomDataset, self).__init__()

        self.x = torch.Tensor(x).flatten()
        self.y = torch.Tensor(y).flatten()
        self.t = torch.Tensor(t).flatten()

        self.u = torch.Tensor(u).flatten()
        self.v = torch.Tensor(v).flatten()
        self.p = torch.Tensor(p).flatten()

        mask = self.t <= 20
        self.x, self.y, self.t = self.x[mask], self.y[mask], self.t[mask]
        self.u, self.v, self.p = self.u[mask], self.v[mask], self.p[mask]

        self.x_size = len(np.unique(self.x))
        self.y_size = len(np.unique(self.y))

    def __len__(self):
        return len(self.x)

    def __getitem__(self, item):
        return self.x[item], self.y[item], self.t[item], self.u[item], self.v[item], self.p[item]


def load_data(path) -> CustomDataset:
    data = scipy.io.loadmat(path)

    u_star = data['U_star']
    p_star = data['p_star']
    t_star = data['t']
    x_star = data['X_star']

    n = x_star.shape[0]
    t = t_star.shape[0]

    x = np.tile(x_star[:, 0:1], (1, t))
    y = np.tile(x_star[:, 1:2], (1, t))
    t = np.tile(t_star, (1, n)).T

    u = u_star[:, 0, :]
    v = u_star[:, 1, :]
    p = p_star

    x = x.flatten()[:, None]
    y = y.flatten()[:, None]
    t = t.flatten()[:, None]

    u = u.flatten()[:, None]
    v = v.flatten()[:, None]
    p = p.flatten()[:, None]

    p -= p.min()

    return CustomDataset(x, y, t, u, v, p)

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__()

        self.activation = nn.Tanh()

        self.lambda1 = nn.Parameter(torch.Tensor([math.log(0.5)]), requires_grad=True)
        self.lambda2 = nn.Parameter(torch.Tensor([math.log(0.1)]), requires_grad=True)

        self.loss_function = nn.MSELoss()

        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, *input_data):
        return self.network(torch.stack(input_data, dim=1))

In [ ]:
import gc


def pde_loss(model: PINN, x, y, t, u, v):
    x.requires_grad = True
    y.requires_grad = True
    t.requires_grad = True

    u_pred, v_pred, p_pred = model(x, y, t).T
    u_t = torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(u_pred), retain_graph=True, create_graph=True)[0]
    u_x = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), retain_graph=True, create_graph=True)[0]
    u_y = torch.autograd.grad(u_pred, y, grad_outputs=torch.ones_like(u_pred), retain_graph=True, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    v_t = torch.autograd.grad(v_pred, t, grad_outputs=torch.ones_like(v_pred), retain_graph=True, create_graph=True)[0]
    v_x = torch.autograd.grad(v_pred, x, grad_outputs=torch.ones_like(v_pred), retain_graph=True, create_graph=True)[0]
    v_y = torch.autograd.grad(v_pred, y, grad_outputs=torch.ones_like(v_pred), retain_graph=True, create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), retain_graph=True, create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), retain_graph=True, create_graph=True)[0]

    p_x = torch.autograd.grad(p_pred, x, grad_outputs=torch.ones_like(p_pred), retain_graph=True, create_graph=True)[0]
    p_y = torch.autograd.grad(p_pred, y, grad_outputs=torch.ones_like(p_pred), retain_graph=True, create_graph=True)[0]

    pde_loss_u = (u_t + torch.exp(model.lambda1) * (u * u_x + v * u_y) + p_x - torch.exp(model.lambda2) * (
            u_xx + u_yy)).pow(2).mean()
    loss_pred_u = (u_pred - u).pow(2).mean()
    # #
    pde_loss_v = (v_t + torch.exp(model.lambda1) * (u * v_x + v * v_y) + p_y - torch.exp(model.lambda2) * (
            v_xx + v_yy)).pow(2).mean()
    loss_pred_v = (v_pred - v).pow(2).mean()
    incompressible_loss = (u_x + v_y).pow(2).mean()
    return loss_pred_u + loss_pred_v + pde_loss_u + pde_loss_v + incompressible_loss

In [ ]:
epoch = 0


def train(model, path):
    epoch = 0
    dataset = load_data(path)
    optimizer = torch.optim.LBFGS(
        model.parameters(),
        lr=1.0,
        max_iter=100000,
        max_eval=100000,
        history_size=50,
        tolerance_grad=1e-5,
        tolerance_change=1.0 * np.finfo(float).eps,
        line_search_fn="strong_wolfe"
    )
    x, y, t = dataset.x.to(device), dataset.y.to(device), dataset.t.to(device)
    u, v, p = dataset.u.to(device), dataset.v.to(device), dataset.p.to(device)

    def closure():
        global epoch

        epoch += 1
        optimizer.zero_grad()
        loss = pde_loss(model, x, y, t, u, v)
        loss.backward()

        print(
            f"Epoch {epoch}, PDE error: {loss.item()}, lambda1: {torch.exp(model.lambda1).item()}, lambda2: {torch.exp(model.lambda2).item()}")
        return loss

    optimizer.step(closure)

In [ ]:
gc.collect()


In [ ]:
model = PINN(3, 3, 20).to(device)
train(model, "/content/cylinder_nektar_wake.mat")

In [ ]:
ds = load_data("./data/cylinder_nektar_wake.mat")
len(ds)

In [ ]:
model = PINN(3, 3, 20)
model.load_state_dict(torch.load("./model.weights", weights_only=True, map_location=device))

In [ ]:
from IPython.display import clear_output


def draw_with_bar(data, bounds, path=None):
    fig, ax = plt.subplots()

    plt.grid(False)
    plt.xlabel("X")
    plt.ylabel("Y")

    gr = ax.imshow(data, cmap='coolwarm', origin="lower", extent=bounds, vmin=-3, vmax=3)
    axis = inset_axes(ax, width="5%", height="100%", loc='upper left', bbox_to_anchor=(1.01, 0, 1, 1),
                      bbox_transform=ax.transAxes, borderpad=0, )
    axis.tick_params(labelright=True, labelleft=False)
    plt.colorbar(gr, cax=axis)
    if path is not None: plt.savefig(path)


def draw_data(path):
    dataset = load_data(path)
    bounds = [dataset.x.min(), dataset.x.max(), dataset.y.min(), dataset.y.max()]

    for iteration, t in enumerate(np.unique(dataset.t)):
        data = dataset[dataset[:][2] == t][5]
        data = data.reshape((dataset.x_size, dataset.y_size))

        draw_with_bar(data, bounds, path)

    import imageio.v2 as imageio

    with imageio.get_writer('animation.gif', mode='I') as writer:
        for i in range(200):
            writer.append_data(imageio.imread(f"./images/{str(i)}.png"))


def draw_p(T, path=None):
    bounds = [1, 8, -2, 2]
    x_steps = 256
    y_steps = 256
    x, y, t = [], [], []
    for i in range(x_steps + 1):
        for j in range(y_steps + 1):
            curr_x = bounds[0] + (bounds[1] - bounds[0]) * (i / x_steps)
            curr_y = bounds[2] + (bounds[3] - bounds[2]) * (j / y_steps)
            x.append(curr_x)
            y.append(curr_y)
            t.append(T)

    x = torch.Tensor(x)
    y = torch.Tensor(y)
    t = torch.Tensor(t)

    pred = model(x, y, t)
    pred = pred[:, 2]
    pred -= pred.min()
    data = pred.reshape((x_steps + 1, y_steps + 1)).T.detach().numpy()

    draw_with_bar(data, bounds, path)


def compute_derivative(f, axis):
    kernel = torch.tensor([-1, 0, 1], dtype=torch.float32).view(1, -1) if axis == 1 else torch.tensor([-1, 0, 1],
                                                                                                      dtype=torch.float32).view(
        -1, 1)
    kernel = kernel / 2.0
    kernel = kernel.to(f.device).unsqueeze(0).unsqueeze(0)

    f = f.unsqueeze(0).unsqueeze(0)
    grad = torch.nn.functional.conv2d(f, kernel, padding=1)
    return grad.squeeze()


def crop_to_same_size(tensor1, tensor2):
    min_h = min(tensor1.shape[0], tensor2.shape[0])
    min_w = min(tensor1.shape[1], tensor2.shape[1])
    return tensor1[:min_h, :min_w], tensor2[:min_h, :min_w]


def draw_vorticity(T, path=None):
    bounds = [1, 8, -2, 2]
    x_steps = 800
    y_steps = 500
    x, y, t = [], [], []
    for i in range(x_steps + 1):
        for j in range(y_steps + 1):
            curr_x = bounds[0] + (bounds[1] - bounds[0]) * (i / x_steps)
            curr_y = bounds[2] + (bounds[3] - bounds[2]) * (j / y_steps)
            x.append(curr_x)
            y.append(curr_y)
            t.append(T)

    x = torch.Tensor(x)
    y = torch.Tensor(y)
    t = torch.Tensor(t)

    x.requires_grad = True
    y.requires_grad = True
    t.requires_grad = True

    u_pred, v_pred, p_pred = model(x, y, t).T
    u_y = torch.autograd.grad(u_pred, y, grad_outputs=torch.ones_like(u_pred), retain_graph=True, create_graph=True)[0]
    v_x = torch.autograd.grad(v_pred, x, grad_outputs=torch.ones_like(v_pred), retain_graph=True, create_graph=True)[0]

    vorticity = v_x - u_y
    vorticity = vorticity.reshape((x_steps + 1, y_steps + 1)).T.detach().numpy()

    draw_with_bar(vorticity, bounds, path)


In [ ]:
draw_vorticity(0.)

In [ ]:
steps_t = 200
min_t, max_t = 0, 5

for iteration in range(steps_t):
    T = min_t + (max_t - min_t) * (iteration / steps_t)
    draw_vorticity(T, path=f"./images/{iteration}.png")

In [25]:
from PIL import Image

frames = []
f = 200
for frame_number in range(0, f):
    frame = Image.open(f"./images/{frame_number}.png")
    frames.append(frame)

frames[0].save(
    'turbulence.gif',
    save_all=True,
    append_images=frames,
    duration=4 * 1000 / f,
    loop=0,
)

In [ ]:
steps_t = 200
min_t, max_t = 0, 5

for iteration in range(steps_t):
    T = min_t + (max_t - min_t) * (iteration / steps_t)
    draw(T, path=f"./images/{iteration}.png")